In [1]:
import requests

API_KEY = 'l9ji35nq6gwbfix7p8olz16q'
BASE_URL = 'https://openapi.etsy.com/v3/application'
url = f"{BASE_URL}/listings/active"

headers = {
    'x-api-key': API_KEY  # API key for authentication
}

# Initialize an empty list to store all listings
all_listings = []

# Loop through 4 pages of 50 listings each (200 total listings)
for offset in range(0, 200, 50):
    params = {
        'keywords': 'painting, art, print',
        'limit': 50,
        'offset': offset,
        'language': 'en-US'  # Filter to English listings
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        listings = response.json()['results']
        all_listings.extend(listings)
        print(f"Retrieved {len(listings)} listings from offset {offset}")
    else:
        print(f"Error at offset {offset}: {response.status_code} - {response.text}")

print(f"Total listings retrieved: {len(all_listings)}")


Retrieved 50 listings from offset 0
Retrieved 50 listings from offset 50
Retrieved 50 listings from offset 100
Retrieved 50 listings from offset 150
Total listings retrieved: 200


In [2]:
import pandas as pd

# Convert listings to a DataFrame
df = pd.DataFrame(all_listings)

In [3]:
# Flatten all list fields to comma-separated strings
df['tags'] = df['tags'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else '')
df['materials'] = df['materials'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else '')
df['production_partners'] = df['production_partners'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else '')
df['skus'] = df['skus'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else '')
df['style'] = df['style'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else '')

In [4]:
# Flatten the price column into individual fields
df['price_amount'] = df['price'].apply(lambda x: x['amount'] / x['divisor'] if isinstance(x, dict) else None)
df['price_currency'] = df['price'].apply(lambda x: x['currency_code'] if isinstance(x, dict) else None)

# Drop the original price column to avoid issues
df = df.drop('price', axis=1)


In [5]:
# Remove rows where who_made is 'someone_else' or 'collective'
df = df[~df['who_made'].str.strip().str.lower().isin(['someone_else', 'collective'])]


In [6]:
print(df['who_made'].unique())


['i_did']


In [7]:
# Apply exclusions based on discoveries made after previously retrieving data
exclusions = ['Nail Art', 'TV Art', 'pentart']

# Filter out unwanted titles
filtered_listings_df = df[
    ~df['title'].str.contains('|'.join(exclusions), case=False)
]
pd.set_option('display.max_columns', None)
filtered_listings_df.head(50)

,listing_id,user_id,shop_id,title,description,state,creation_timestamp,created_timestamp,ending_timestamp,original_creation_timestamp,last_modified_timestamp,updated_timestamp,state_timestamp,quantity,shop_section_id,featured_rank,url,num_favorers,non_taxable,is_taxable,is_customizable,is_personalizable,personalization_is_required,personalization_char_count_max,personalization_instructions,listing_type,tags,materials,shipping_profile_id,return_policy_id,processing_min,processing_max,who_made,when_made,is_supply,item_weight,item_weight_unit,item_length,item_width,item_height,item_dimensions_unit,is_private,style,file_data,has_variations,should_auto_renew,language,taxonomy_id,production_partners,skus,views,price_amount,price_currency
0,1567545027,82148,5293743,Framed Mini Art Black Witch Kitty Mini Gold B...,This is a small high quality canvas giclee pri...,active,1730931488,1730931488,1741299488,1694819594,1730931488,1730931488,1729277517,7,27567026.0,-1,https://www.etsy.com/listing/1567545027/framed...,2589,False,True,False,False,False,NaN,None,physical,"gallery, canvas print, halloween cat, hallowee...",,2.195195e+11,1.082366e+12,1.0,2.0,i_did,2020_2024,False,5.0,oz,7.0,5.0,2.0,in,False,,,False,False,en-US,1027,,,13986,22.00,USD
1,1254888432,141713720,17199051,Field of Fireflies - Summer Evening Sunset Wat...,Colorful Sunset Watercolor Illustration - Fire...,active,1730931480,1730931480,1741299480,1657916511,1730931480,1730931480,1674253126,2996,26591064.0,-1,https://www.etsy.com/listing/1254888432/field-...,578,False,True,False,False,False,NaN,None,physical,"fireflies at night, forest of fireflies, firef...","archival quality color ink, giclee print paper...",7.804526e+10,1.079825e+12,6.0,8.0,i_did,made_to_order,False,NaN,None,NaN,NaN,NaN,None,False,,,True,True,en-US,121,"{'production_partner_id': 715585, 'partner_nam...",,7424,34.00,USD
2,985700215,88171156,13148149,PRINT***Anakin&#39;s Betrayal 8.5”x11” or 11&q...,A powerful representation of the betrayal of A...,active,1730931476,1730931476,1741299476,1616389409,1730931476,1730931476,1616389409,76,26317161.0,3,https://www.etsy.com/listing/985700215/printan...,608,False,True,False,False,False,NaN,None,physical,"Star Wars inspired, Duel of the Fates, Revenge...","Archival Ink, Velvet Fine Art Paper",1.760458e+11,1.139160e+12,7.0,10.0,i_did,2005_2009,False,2.0,oz,12.0,9.0,0.5,in,False,,,True,False,en-US,121,,,4818,20.00,USD
3,1083643600,46249978,9331856,"watercolor painting from photo, custom dog por...",This amazing watercolor painting from a photo ...,active,1730931453,1730931453,1741299453,1633551728,1730931453,1730931453,1730665515,183,40218672.0,-1,https://www.etsy.com/listing/1083643600/waterc...,6600,False,True,True,True,False,256.0,"After you purchase the item, please send us an...",physical,"photo into digital, gift for her him, Wall art...","Wrapped Canvas, Fine Art Paper, Wood Framed Po...",7.568079e+10,1.082366e+12,1.0,3.0,i_did,2020_2024,False,2.0,oz,10.0,5.0,2.0,in,False,,,True,True,en-US,2078,,,347159,25.00,USD
4,1818461473,104799210,14740712,"For Safekeeping print (couple version), Childl...",This is a detailed print of my original painti...,active,1730931450,1730931450,1741299450,1730384868,1730931450,1730931450,1730384868,8,49301410.0,-1,https://www.etsy.com/listing/1818461473/for-sa...,0,False,True,False,False,False,NaN,None,physical,"pregnancy loss, infant loss, Christianity, Chr...","paper, paint",7.711787e+10,1.104749e+12,1.0,4.0,i_did,2020_2024,False,4.0,oz,9.0,6.0,0.1,in,False,,,True,True,en-US,121,,,26,15.00,USD
5,1659433562,882799997,49121351,By His Wounds | Digital Download | Jesus Chris...,"Instant Access, Eternal Message: Dive into the...",active,1730931449,1730931449,1741299449,1707253791,1730931449,1730931449,1721936538,886,48198823.0,9,https://www.etsy.com/listing/1659433562/by-his...,456,False,True,False,False,False,NaN,None,download,"Jesus, Christian art, lds art, Jesus Christ, l...",Digital Art,NaN,1.000000e+00,NaN,NaN,i_did,2020_2024,False

In [8]:
# Get the number of rows and columns
num_rows, num_columns = filtered_listings_df.shape

# Print the number of listings
print(f"Number of listings remaining: {num_rows}")


Number of listings remaining: 167


In [9]:
# Check for any remaining list-type values
for column in filtered_listings_df.columns:
    print(f"{column}: {df[column].apply(lambda x: isinstance(x, list)).sum()} lists remaining")

listing_id: 0 lists remaining
user_id: 0 lists remaining
shop_id: 0 lists remaining
title: 0 lists remaining
description: 0 lists remaining
state: 0 lists remaining
creation_timestamp: 0 lists remaining
created_timestamp: 0 lists remaining
ending_timestamp: 0 lists remaining
original_creation_timestamp: 0 lists remaining
last_modified_timestamp: 0 lists remaining
updated_timestamp: 0 lists remaining
state_timestamp: 0 lists remaining
quantity: 0 lists remaining
shop_section_id: 0 lists remaining
featured_rank: 0 lists remaining
url: 0 lists remaining
num_favorers: 0 lists remaining
non_taxable: 0 lists remaining
is_taxable: 0 lists remaining
is_customizable: 0 lists remaining
is_personalizable: 0 lists remaining
personalization_is_required: 0 lists remaining
personalization_char_count_max: 0 lists remaining
personalization_instructions: 0 lists remaining
listing_type: 0 lists remaining
tags: 0 lists remaining
materials: 0 lists remaining
shipping_profile_id: 0 lists remaining
return_po

In [10]:
import sqlite3
# Connect to (or create) the SQLite database
conn = sqlite3.connect('etsy_data.db')

# Save the DataFrame to a SQL table
df.to_sql('etsy_listings', conn, if_exists='replace', index=False)

print("Data successfully saved to SQLite database.")
conn.close()


Data successfully saved to SQLite database.


In [11]:
# Extract unique shop IDs from the df to retrieve corresponding shop data from API
shop_ids = df['shop_id'].unique()
print(f"Total unique shops: {len(shop_ids)}")


Total unique shops: 141


In [12]:
shop_data = []

for shop_id in shop_ids:
    shop_url = f"{BASE_URL}/shops/{shop_id}"
    response = requests.get(shop_url, headers=headers)

    if response.status_code == 200:
        shop_data.append(response.json())
    else:
        print(f"Error retrieving shop {shop_id}: {response.status_code}")

print(f"Retrieved data for {len(shop_data)} shops.")

Retrieved data for 141 shops.


In [13]:
shop_df = pd.DataFrame(shop_data)
shop_df.head()

,shop_id,shop_name,user_id,create_date,created_timestamp,title,announcement,currency_code,is_vacation,vacation_message,sale_message,digital_sale_message,update_date,updated_timestamp,listing_active_count,digital_listing_count,login_name,accepts_custom_requests,vacation_autoreply,url,image_url_760x100,num_favorers,languages,icon_url_fullxfull,is_using_structured_policies,has_onboarded_structured_policies,include_dispute_form_link,is_direct_checkout_onboarded,is_etsy_payments_onboarded,is_opted_in_to_buyer_promise,is_calculated_eligible,is_shop_us_based,transaction_sold_count,shipping_from_country_iso,shop_location_country_iso,policy_welcome,policy_payment,policy_shipping,policy_refunds,policy_additional,policy_seller_info,policy_update_date,policy_has_private_receipt_info,has_unstructured_policies,policy_privacy,review_average,review_count
0,5293743,Posiez,82148,1210104437,1210104437,"Affordable prints, small gift ideas,\nOriginal...","NO AI. I have for you, lots of perfect little ...",USD,False,Follow me on social media https://linktr.ee/qu...,thank you so much! :)\r\n\r\nif there is ever ...,"If you have download issues, please send me an...",1730931488,1730931488,255,71,Posiez,False,Please send me an email to queenieposiez@gmail...,https://www.etsy.com/shop/Posiez,https://i.etsystatic.com/iusb/295d7e/72604437/...,13093,[en-US],https://i.etsystatic.com/isla/d528a7/17680440/...,True,True,False,True,True,False,True,True,19551,None,US,None,SECURE OPTIONS\nEtsy keeps your payment inform...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT CANCELLATIONS\nRequest a cance...,,,1676560370,False,False,,4.9736,1517.0
1,17199051,AndreaHelmArtwork,141713720,1519749811,1519749811,Art Prints and Paintings by Andrea Helm,LAST DAY to order Custom Room Portraits for 20...,USD,False,None,Thank you for your purchase! I appreciate eve...,None,1730931481,1730931481,730,0,u96vynsk,False,None,https://www.etsy.com/shop/AndreaHelmArtwork,None,497,[en-US],https://i.etsystatic.com/isla/5428be/36469401/...,True,True,False,True,True,True,True,True,871,US,US,None,SECURE OPTIONS\nEtsy keeps your payment inform...,PROCESSING TIME\nThe time I need to prepare an...,"I DON'T ACCEPT RETURNS, EXCHANGES, OR CANCELLA...",,,1574800364,False,False,,4.9375,64.0
2,13148149,EnchantedStudioCo,88171156,1466626217,1466626217,Custom Illustrations + Prints + Fantasy + Whimsy,None,USD,False,"Hello, everyone! I will be closing up shop for...",Thank you for purchasing from Enchanted Studio...,None,1730931476,1730931476,96,0,enchantedstudioco,True,"Hello, everyone! I will be closing up shop for...",https://www.etsy.com/shop/EnchantedStudioCo,None,1991,[en-US],https://i.etsystatic.com/isla/602241/32137985/...,False,False,False,True,True,True,True,True,3570,US,US,None,None,None,None,None,None,0,False,False,None,5.0000,147.0
3,9331856,BeautifulPrint,46249978,1396980481,1396980481,watercolor painting from photo,"Introducing BeautifulPrint, your go-to shop fo...",USD,False,Hello and thanks for visiting! I’m working on ...,"Dear BeautifulPrint shoppers, \r\n\r\nWe reall...","Dear BeautifulPrint shoppers, \r\n\r\nWe reall...",1730931454,1730931454,132,0,komanast1,True,Hello and thanks for your message! I’m working...,https://www.etsy.com/shop/BeautifulPrint,https://i.etsystatic.com/iusb/39e51a/63091709/...,5504,"[en-US, es]",https://i.etsystatic.com/isla/45c539/61822950/...,False,False,False,True,True,True,True,True,39160,US,US,None,None,None,None,None,None,1674498796,False,False,None,4.8183,1387.0
4,14740712,PaigePayneCreations,104799210,1489107752,1489107752,By Paige Payne,I am a watercolor and artist painting explorin...,USD,False,My shop is currently closed for Christmas. I w...,Thank you so much for supporting me and my hus...,Enjoy this digital download! Thank you so much...,1730931450,1730931450,319,3,paigepayne96,False,None,https://www.etsy.com/shop/PaigePayneCreations,None,2697,[en-US],https://i.etsystatic.com/isla/b39da4/45148611/...,True,True,False,True,True

In [14]:
# Check if any columns contain lists or dictionaries
for column in shop_df.columns:
    if shop_df[column].apply(lambda x: isinstance(x, (list, dict))).any():
        print(f"Column '{column}' contains lists or dictionaries.")


Column 'languages' contains lists or dictionaries.


In [15]:
# Convert the lists in 'languages' to comma-separated strings
shop_df['languages'] = shop_df['languages'].apply(
    lambda x: ', '.join(x) if isinstance(x, list) else ''
)

In [16]:
conn = sqlite3.connect('etsy_data.db')
shop_df.to_sql('etsy_shops', conn, if_exists='replace', index=False)
conn.close()
print("Shop data saved successfully.")


Shop data saved successfully.


In [17]:
import requests
import pandas as pd


# Extract listing IDs from the existing DataFrame
listing_ids = df['listing_id'].unique()  # Get unique listing IDs

print(listing_ids) 

# Prepare to collect reviews
reviews_data = []

# Base URL for reviews retrieval
BASE_URL = 'https://openapi.etsy.com/v3/application'


headers = {
    'x-api-key': API_KEY 
}

# Retrieve reviews for each listing ID
for listing_id in listing_ids:
    review_url = f"{BASE_URL}/listings/{listing_id}/reviews"
    response = requests.get(review_url, headers=headers)

    if response.status_code == 200:
        reviews = response.json().get('results', [])
        reviews_data.extend(reviews)  # Collect all reviews
    else:
        print(f"Error retrieving reviews for listing {listing_id}: {response.status_code}")

# Print total reviews retrieved
print(f"Total reviews retrieved: {len(reviews_data)}")

# Convert the reviews data into a DataFrame
reviews_df = pd.DataFrame(reviews_data)

# Display the first few rows of the reviews DataFrame
print(reviews_df.head(50))


[1567545027 1254888432  985700215 1083643600 1818461473 1659433562
 1786267074 1786141315 1398011570 1807546702 1724088883  874088246
 1821745117 1594976623 1729608168 1749336649 1807549094 1806651920
 1674608982  400586043 1728021974 1797054647 1821744541 1013775926
 1349321977 1779797604 1793994849 1770149960 1807548228 1346332774
 1548945348 1807547654 1807548080 1807548018 1380166648 1807547664
 1574076683 1559713902 1614364876 1003400061  840246218 1745003848
 1695131762 1607664682 1731975754 1677788293 1524643420 1741898323
 1675999838 1210168541 1602746885  686271207  913919274 1476335263
 1676021988 1757546131 1757546129 1499653846  596176269 1536065951
 1660810132 1757546511 1599389212 1340145706 1742072161  969968023
 1321597296 1414621894 1743343064 1721120385 1716033190 1794745340
 1260384128 1463862992 1803951228 1515541359 1320989324 1493943137
 1532660024 1546764501  732336206  711665597 1667990590 1561698798
 1807546770 1552589219 1395256270 1010291881 1414645102 114379

In [19]:
#Sentiment analysis on reviews
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Perform sentiment analysis on each review
reviews_df['sentiment'] = reviews_df['review'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [20]:
reviews_df.head(50)

,shop_id,listing_id,rating,review,language,image_url_fullxfull,create_timestamp,created_timestamp,update_timestamp,updated_timestamp,sentiment
0,5293743,1567545027,5,Absolutely beautiful and came shockingly quick...,en,None,1730685105,1730685105,1730688918,1730688918,0.8830
1,5293743,1567545027,5,It was just as pictured. Love it.,en,None,1729627010,1729627010,1729631132,1729631132,0.6369
2,5293743,1567545027,5,Perfect! Love these cute cats.,en,None,1729524476,1729524476,1729527648,1729527648,0.9041
3,5293743,1567545027,5,Got this as a gift for my mom because it close...,en,None,1729339383,1729339383,1729343230,1729343230,0.8118
4,5293743,1567545027,5,Great gift! Shipping was fast.,en,None,1728398981,1728398981,1728402224,1728402224,0.8070
5,5293743,1567545027,5,Purrfect portrait of a beloved hex kitten 😻,en,None,1727991388,1727991388,1727994716,1727994716,0.7430
6,5293743,1567545027,5,It is beautiful and I love the attention to de...,en,None,1727144863,1727144863,1727148506,1727148506,0.8442
7,5293743,1567545027,5,Ahh! This is SO cute! I can’t wait to hang it ...,en,None,1726259207,1726259207,1726263081,1726263081,0.9430
8,5293743,1567545027,5,Super cute!!! I can’t wait to gift it to my fr...,en,None,1726253863,1726253863,1726257799,1726257799,0.9345
9,5293743,1567545027,5,Beautiful! Thank you! Will go perfect with my ...,en,None,1725375060,1725375060,1725378665,1725378665,0.8930


In [21]:
# Connect to the SQLite database
conn = sqlite3.connect('etsy_data.db')

# Save reviews to SQLite
reviews_df.to_sql('etsy_reviews', conn, if_exists='replace', index=False)

# Close the connection
conn.close()
print("Reviews saved successfully.")


Reviews saved successfully.
